In [1]:
# 직접 코딩해보자

import sys, os
sys.path.append(os.pardir)
import numpy as np

# 실습의 common에 있는 function들을 가져와 쓴다 
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

In [5]:
# class 만들어보자
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)  # 표준 정규분포
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        print("y.shape", y.shape)  # W값 하나에 대해 loss는 두 번 불리게 된다. f(x+h)에 한번, f(x-h)에서 한번 (여기서 x는 W값이다)
        loss = cross_entropy_error(y, t)
        
        return loss
    
x = np.array([.6, .9])
t = np.array([0, 0, 1])

net = simpleNet()

f = lambda k: net.loss(x, t)
"""
1) 초기 W값으로 예측을 해본다 predict
2) 예측값(=score)을 확률로 바꾼다. softmax 사용. 따라서 y.shape = (3,)
3) 마지막으로 loss 값을 계산해본다. 
"""
dW = numerical_gradient(f, net.W)

print(dW)

y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
y.shape (3,)
[[ 0.01637896  0.01543468 -0.03181364]
 [ 0.02456844  0.02315202 -0.04772046]]


1. 위에서는 6개의 W 기울기 값을 알아내기만 했다
2. 이제 이걸로 학습을 시켜야 한다


# 2층 신경망을 만들자

## 우선은 클래스부터 만들자

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # Weight의 초기화 작업
        self.params = {}  # 딕셔너리로 구현한다
        
        # Weight, bias 들의 크기는 input_size, hidden_size, output_size에 영향을 받는다
        # Weight에는 표준정규분포에 0.01을 곱해서 더 작은 값으로 만들기
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)  
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # 입력 데이터 x를 받아 값을 예측한 다음 정답과 비교하여 loss를 계산한다
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y,t)
    
    def accuracy(self, x, t)
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
        def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0] 
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads